In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

from dotenv import load_dotenv

sys.path.insert(0, "..")

load_dotenv()

True

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import duckdb
import hvplot.polars
import numpy as np
import polars as pl

In [5]:
from tsdb_benchmarks.settings import SETTINGS

In [14]:
db = duckdb.connect(SETTINGS.results_directory / "results.db")

db.execute("select * from metric").pl().select("ts", "cpu_percent").hvplot(x="ts")

:Curve   [ts]   (cpu_percent)